In [1]:
import numpy as np
import pandas as pd

In [2]:
!ls

 stage_2_sample_submission.csv	 stage_2_train.csv
 stage_2_test_images		'Update Dataset Statistics.ipynb'


## ICH or Naw

In [3]:
df = pd.read_csv('stage_2_train.csv')
df["Image"] = df["ID"].str.slice(stop=12)
df["Diagnosis"] = df["ID"].str.slice(start=13)

duplicates_to_remove = [
    56346, 56347, 56348, 56349, 56350, 56351, 1171830, 1171831, 1171832, 1171833, 1171834,
    1171835, 3705312, 3705313, 3705314, 3705315, 3705316, 3705317, 3842478, 3842479, 
    3842480, 3842481, 3842482, 3842483
]

df = df.drop(index=duplicates_to_remove)
df = df.reset_index(drop=True)

df = df.loc[:, ["Label", "Diagnosis", "Image"]]
df = df.set_index(['Image', 'Diagnosis']).unstack(level=-1)

In [4]:
print(df['Label']['any'].value_counts())

0    644870
1    107933
Name: any, dtype: int64


In [5]:
df['Label']['any'].value_counts()/len(df.index)

0    0.856625
1    0.143375
Name: any, dtype: float64

The dataset distribution between negative and positive samples stayed the same (85.6% negative and 14.3% postitive)

## Diagnoses Distribution

In [6]:
diagnoses = df['Label'].sum()/df['Label']['any'].sum()
diagnoses = diagnoses.drop('any')

import matplotlib.pyplot as plt
plt.bar(diagnoses.keys(), height = diagnoses.values)
plt.xlabel('Diagnosis')
plt.ylabel('Probability')
plt.xticks(rotation='vertical')
plt.show()

<Figure size 640x480 with 1 Axes>

In [7]:
df.sum()

       Diagnosis       
Label  any                 107933
       epidural              3145
       intraparenchymal     36118
       intraventricular     26205
       subarachnoid         35675
       subdural             47166
dtype: int64

## Multiple Diagnosis

In [8]:
df[df.sum(axis=1)>2]

Label                                                          \
Diagnosis      any epidural intraparenchymal intraventricular subarachnoid   
Image                                                                        
ID_0002a38ad     1        0                0                0            1   
ID_0005d340e     1        0                0                1            1   
ID_00094b19b     1        0                1                0            1   
ID_000a050f3     1        0                0                0            1   
ID_000d66d9e     1        0                0                0            1   
...            ...      ...              ...              ...          ...   
ID_fff9bb57a     1        1                1                1            0   
ID_fffa58f70     1        0                1                1            0   
ID_fffb124e6     1        0                0                0            1   
ID_fffc60817     1        0                1                1            0   
ID_fffe2edb8     1        0                1                1            0   

                       
Diagnosis    subdural  
Image                  
ID_0002a38ad        1  
ID_0005d340e        0  
ID_00094b19b        0  
ID_000a050f3        1  
ID_000d66d9e        1  
...               ...  
ID_fff9bb57a        0  
ID_fffa58f70        1  
ID_fffb124e6        1  
ID_fffc60817        0  
ID_fffe2edb8        0  

[32074 rows x 6 columns]

In [9]:
len(df[df.sum(axis=1)>1]) #number of images with any diagnosis.

107933

In [10]:
len(df[df.sum(axis=1)==6]) #number of images with all 6 kinds of hemorrhage

23